In [4]:
import hashlib
import bisect


class ConsistentHashing:
    def __init__(self, nodes=None, replicas=3):
        self.replicas = replicas
        self.ring = dict()
        self.sorted_keys = []

        if nodes:
            for node in nodes:
                self.add_node(node)

    def _hash(self, key):
        return int(hashlib.md5(key.encode("utf-8")).hexdigest(), 16)

    def add_node(self, node):

        for i in range(self.replicas):

            virtual_node_key = f"{node}:{i}"
            key_hash = self._hash(virtual_node_key)

            self.ring[key_hash] = node

            bisect.insort(self.sorted_keys, key_hash)

    def remove_node(self, node):

        for i in range(self.replicas):
            virtual_node_key = f"{node}:{i}"
            key_hash = self._hash(virtual_node_key)

            del self.ring[key_hash]
            self.sorted_keys.remove(key_hash)

    def get_node(self, key):

        if not self.ring:
            return None

        key_hash = self._hash(key)

        idx = bisect.bisect(self.sorted_keys, key_hash)

        if idx == len(self.sorted_keys):
            idx = 0

        return self.ring[self.sorted_keys[idx]]


ch = ConsistentHashing(["cache-A", "cache-B", "cache-C"], replicas=3)

users = ["user_123", "user_456", "user_789", "user_999", "user_000"]

print("--- Distribuição Inicial ---")
initial_map = {}
for u in users:
    node = ch.get_node(u)
    initial_map[u] = node
    print(f"User {u} -> {node}")

print("\n--- Removendo 'cache-A' (Falha) ---")
ch.remove_node("cache-A")

for u in users:
    new_node = ch.get_node(u)
    if new_node != initial_map[u]:
        print(f"User {u} MOVIDO de {initial_map[u]} para {new_node}")
    else:
        print(f"User {u} manteve-se em {new_node}")

print("\n--- Adicionando 'cache-D' (Escalando) ---")
ch.add_node("cache-D")

--- Distribuição Inicial ---
User user_123 -> cache-C
User user_456 -> cache-B
User user_789 -> cache-B
User user_999 -> cache-B
User user_000 -> cache-B

--- Removendo 'cache-A' (Falha) ---
User user_123 manteve-se em cache-C
User user_456 manteve-se em cache-B
User user_789 manteve-se em cache-B
User user_999 manteve-se em cache-B
User user_000 manteve-se em cache-B

--- Adicionando 'cache-D' (Escalando) ---
